In [7]:
import tensorflow as tf 
Elytis_model = tf.keras.models.load_model('Elytis_stateful_model')

In [2]:
# read the dataset
with open("cleaned_combined_Elytis.txt") as f:
    data = f.read() 

def clean_text(text):
    import re 
    """
    Removes all non-Greek characters and numbers from the given text.

    Args:
    text (str): The input text to process.

    Returns:
    str: The cleaned text containing only Greek characters.
    """
    # Define a regular expression pattern that matches Greek characters (uppercase and lowercase)
    # and spaces (optional if you want to preserve spaces)
    pattern = r'[^\u0370-\u03FF\u1F00-\u1FFF\s]'
    
    # Use re.sub to replace all characters that don't match the pattern with an empty string
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text
data = clean_text(data) 
n_distinct_characters = len(set(data)) 


text_vec_layer = tf.keras.layers.TextVectorization(
    split="character", standardize="lower")
text_vec_layer.adapt([data])
encoded = text_vec_layer([data])[0]
encoded -= 2  # drop tokens 0 (pad) and 1 (unknown), which we will not use
n_tokens = text_vec_layer.vocabulary_size() - 2 # number of distinct chars = 39
dataset_size = len(encoded)

print(n_tokens) 

37


In [3]:
def next_char(text, temperature=1):
    y_proba = Elytis_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [4]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [5]:
extend_text("καλώς εχόντων τω", 10, .1)  

1/1 [==============================] - 0s 35ms/step


KeyboardInterrupt: 